In [49]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [50]:
import time
import json
import os
import random
from typing import Dict, Iterator, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from loguru import logger
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold

from nam.config.default import defaults
from nam.types import Config
from nam.utils.args import parse_args
from nam.data.base import NAMDataset

In [51]:
config = defaults()
config

namespace(device='cpu',
          logdir='logs',
          lr=0.01,
          batch_size=1024,
          l2_regularization=0.0,
          output_regularization=0.0,
          decay_rate=0.995,
          dropout=0.5,
          feature_dropout=0.0,
          data_split=1,
          seed=1377,
          num_basis_functions=1000,
          units_multiplier=2,
          cross_val=False,
          max_checkpoints_to_keep=1,
          save_checkpoint_every_n_epochs=10,
          n_models=1,
          num_splits=3,
          fold_num=1,
          activation='exu',
          regression=False,
          debug=False,
          shallow=False,
          use_dnn=False,
          early_stopping_epochs=60,
          n_folds=5)

## GALLUP Data

In [52]:
features_columns = ["income_2", "WP1219", "WP1220", "weo_gdpc_con_ppp"]
targets_column = ["country"]
weights_column = ["wgt"]

In [53]:
dataset = NAMDataset(config=config,
                    csv_file='data/GALLUP.csv',
                    features_columns=features_columns,
                    targets_column=targets_column,
                    weights_column=weights_column)
dataset

### K-fold

In [54]:
for train_dl, test_dl in dataset.data_loaders():
    print(next(iter(train_dl)), next(iter(test_dl)))

Fold((1,)), train: 1072647, test: 268162
[tensor([[22196,     0,    34,   627],
        [31624,     1,    11,   604],
        [37435,     0,     7,   340],
        ...,
        [    0,     1,    16,   247],
        [34044,     1,    31,   398],
        [69811,     1,    68,  1038]]), tensor([[254198],
        [161876],
        [ 69969],
        ...,
        [130225],
        [139738],
        [ 18629]]), tensor([[ 90],
        [148],
        [ 16],
        ...,
        [127],
        [108],
        [133]])] [tensor([[ 6944,     1,    31,   294],
        [62006,     0,    34,   571],
        [ 5782,     0,     8,   319],
        ...,
        [58303,     1,    16,   344],
        [77505,     1,    33,   960],
        [64443,     0,    16,   750]]), tensor([[147747],
        [ 99166],
        [ 20756],
        ...,
        [119298],
        [278523],
        [196549]]), tensor([[60],
        [63],
        [33],
        ...,
        [15],
        [50],
        [70]])]
Fold((2,)), train: 10

## Housing Data

In [55]:

features_columns = [
    "longitude", "latitude", "housing_median_age", "total_rooms",
    "total_bedrooms", "population", "households", "median_income"
]
targets_column = ["median_house_value"]

In [56]:
dataset = NAMDataset(config=config,
                    csv_file='data/housing.csv',
                    features_columns=features_columns,
                    targets_column=targets_column)
dataset

In [57]:
for train_dl, test_dl in dataset.data_loaders():
    print(next(iter(train_dl)), next(iter(test_dl)))

Fold((1,)), train: 16512, test: 4128
[tensor([[  563,   146,    51,  ...,  1179,   383,  1005],
        [  629,   154,    34,  ...,   702,   252,  5761],
        [  550,   172,    29,  ...,  1960,   368,  5794],
        ...,
        [  552,   153,    51,  ...,   382,   137, 12927],
        [  550,   164,    32,  ...,  1519,   596,  6786],
        [  558,   148,    26,  ...,  2358,   954,   575]]), tensor([[ 745],
        [ 765],
        [ 993],
        ...,
        [3841],
        [2245],
        [ 460]])] [tensor([[  559,   139,    32,  ...,  3481,  1175,  3967],
        [  599,   120,    23,  ...,  2907,   797,  7546],
        [  175,   500,    41,  ...,   619,   197,  4159],
        ...,
        [  160,   521,    46,  ...,   770,   341,  6004],
        [  489,   336,    16,  ...,  1883,   617,   785],
        [  199,   483,    14,  ...,  2355,   713, 12440]]), tensor([[1021],
        [1655],
        [ 385],
        ...,
        [ 992],
        [ 110],
        [3620]])]
Fold((2,)), t